Activity 3

In [1]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain_google_vertexai import ChatVertexAI
import requests
from typing import Tuple, List
import csv

MODEL_NAME = "gemini-2.0-flash-001"
MODEL_KWARGS = {
    "temperature": 0.28,
    "max_output_tokens": 8192,
    "top_p": 0.95,
    "top_k": None,}

MODEL = ChatVertexAI(model_name=MODEL_NAME, **MODEL_KWARGS)

checkpointer = InMemorySaver()

def get_weather(city_name: str) -> str:
    """
    Fetches comprehensive weather information for a specified city using the wttr.in service.
    """
    weather_url = f"https://wttr.in/{city_name}?format=%C+%t+%w+%h+%p+%P+%m+%M+%S"
    response = requests.get(weather_url)
    if response.status_code != 200:
        return "Weather information unavailable at the moment."
    
    print("Weather Agent used")
    return response.text


def get_resorts(city_name: str, csv_path: str = "C:/Users/richard.c.ferrer/Agents/resorts_data.csv") -> List[List[str]]:
    """
    Retrieves resorts from a CSV file filtered by the specified city name.

    Returns:
    List of [Resort Name, Location]
    """
    resorts = []
    try:
        with open(csv_path, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if city_name.lower() in row["Location"].lower():
                    resorts.append([row["Resort"], row["Location"]])
    except FileNotFoundError:
        print("CSV file not found.")
    except KeyError:
        print("CSV headers are incorrect. Expected: 'Resort' and 'Location'.")

    return resorts

def get_bestresorts(city_name: str) -> Tuple[str, List[List[str]]]:
    """
    Combines weather data and resort list into a single tool response.
    """
    weather_data = get_weather(city_name)
    resorts_data = get_resorts(city_name)
    return weather_data, resorts_data

bestresorts_agent = create_react_agent(
    model=MODEL,
    tools=[get_bestresorts],
    prompt="""
    Check the weather and get the best resorts using your tool.
    """,
    name="bestresorts_agent",
    checkpointer=checkpointer
)



In [2]:
response = bestresorts_agent.invoke(
    {
        "messages": [{"role": "user", "content": "What are some good resorts in Calamba and how is the weather there right now?"}]
    },
    config={"configurable": {"thread_id": "1"}},
)
print(response['messages'][-1].content)


Weather Agent used
OK. The weather in Calamba is patchy rain nearby, +32°C, ↓7km/h, 71%, 0.0mm, 1010hPa, 🌔 9 05:26:06. Some resorts in Calamba are The Cliffhouse Calamba, Casa Palmera Royale, La Vista Pansol Resort, 88 Hotspring Resort, and Hannah Mountain Resort and Events Place.

